In [1]:
import sys
if "../" not in sys.path:
    sys.path.insert(0, "../")

import cobra.test
import escher
import numpy as np

from ecoli.library import fba_gd


# Test case: OAA -> Asp -> Asn, Lys, Met, Thr

### Read full E. coli model from cobra, and extract a subset of reaction stoichiometries to work with

In [2]:
full_model = cobra.test.create_test_model("ecoli")  # iJO1366

# Reaction structures follows the knowledge base.
reactions = []
for reaction in full_model.reactions:
    if reaction.id in [
        "ASPTA",  # -> Asp
#         "ASNN",  # -> Asn. But in iJO1366 this reaction is irreversible, Asn -> Asp only.
        "ASPK",
        "ASAD",

        "DHDPS",
        "DHDPRy",
        "THDPS",
        "SDPTA",
        "SDPDS",
        "DAPE",
        "DAPDC",  # -> Lys

        "HSDy",
        "HSK",
        "THRS",  # -> Thr

        "HSST",
        "SHSL1",
        "CYSTL",
        "METS",  # -> Met
    ]:
        reactions.append({"reaction id": reaction.id,
                          "stoichiometry": {metabolite.id: coeff
                                            for metabolite, coeff in reaction.metabolites.items()},
                          "is reversible": reaction.lower_bound < 0})

exchanges = ['5mthf_c', 'adp_c', 'akg_c', 'atp_c', 'co2_c', 'coa_c', 'cys__L_c',
             'glu__L_c', 'h2o_c', 'h_c', 'nadp_c', 'nadph_c', 'nh4_c', 'oaa_c', 'pi_c',
             'pyr_c', 'succ_c', 'succoa_c', 'thf_c']

production_targets = {
#     "asn__L_c": 0.6, 
    "asp__L_c": 1.0,
    "lys__L_c": 0.9,
    "met__L_c": 0.5,
    "thr__L_c": 2.
}


### Set up the structure of the FBA problem

In [3]:
fba1 = fba_gd.GradientDescentFba(
    reactions=reactions,
    exchanges=exchanges,
    target_metabolites=production_targets.keys(),
)
fba1.add_objective("homeostatic",
                   fba_gd.TargetDmdtObjective(fba1.network, production_targets.keys()))

### Solve the problem, passing in current objectives

In [4]:
soln1 = fba1.solve({"homeostatic": production_targets})

# Steady-state residual is a measure of how far the solution is from steady-state
print(f"RMSE = {np.sqrt(np.sum(np.square(soln1.ss_residual)))}")

RMSE = 1.567938595257497e-07


### The solution object has solved values for reaction velocities, and rates of change (dm/dt) of all molecules

- Objective molecules have dm/dt close to their objective values
- Exchange molecules' dm/dt are as needed to support production of the objectives
- Internal (intermediate) molecules have dm/dt close to zero (i.e. steady state)

In [5]:
print("Velocities")
for reaction_id in sorted(reaction["reaction id"] for reaction in reactions):
    print(f"  {reaction_id:>10}:   {soln1.velocities[reaction_id]:+.2f}")

print()
print("Production Targets")
for molecule in sorted(production_targets):
    print(f"  {molecule:>10}:   {soln1.dm_dt[molecule]:.2f} / {production_targets[molecule]:.2f}")

print()
print("Exchanges")
for molecule in sorted(exchanges):
    print(f"  {molecule:>10}:   {soln1.dm_dt[molecule]:+.2f}")

print()
print("Intermediates")
for molecule in fba1.network.molecule_ids():
    if molecule not in production_targets and molecule not in exchanges:
        print(f"  {molecule:>10}:   {soln1.dm_dt[molecule]: .2e}")

escher.Builder(map_json='FBAgd-demo-eschermap.json',
               menu='zoom',
               never_ask_before_quit=True,
               reaction_data=soln1.velocities,
               reaction_styles=['color', 'size', 'text', 'abs'],
               metabolite_data=soln1.dm_dt,
               metabolite_styles=['color', 'text'],
               metabolite_scale_preset='GeGaRd',
              )


Velocities
        ASAD:   -3.40
        ASPK:   +3.40
       ASPTA:   -4.40
       CYSTL:   +0.50
       DAPDC:   +0.90
        DAPE:   +0.90
      DHDPRy:   +0.90
       DHDPS:   +0.90
        HSDy:   -2.50
         HSK:   +2.00
        HSST:   +0.50
        METS:   +0.50
       SDPDS:   +0.90
       SDPTA:   -0.90
       SHSL1:   +0.50
       THDPS:   +0.90
        THRS:   +2.00

Production Targets
    asp__L_c:   1.00 / 1.00
    lys__L_c:   0.90 / 0.90
    met__L_c:   0.50 / 0.50
    thr__L_c:   2.00 / 2.00

Exchanges
     5mthf_c:   -0.50
       adp_c:   +5.40
       akg_c:   +5.30
       atp_c:   -5.40
       co2_c:   +0.90
       coa_c:   +1.40
    cys__L_c:   -0.50
    glu__L_c:   -5.30
       h2o_c:   -2.50
         h_c:   -3.80
      nadp_c:   +6.80
     nadph_c:   -6.80
       nh4_c:   +0.50
       oaa_c:   -4.40
        pi_c:   +5.40
       pyr_c:   -0.40
      succ_c:   +1.40
    succoa_c:   -1.40
       thf_c:   +0.50

Intermediates
     aspsa_c:    1.05e-07
     4pasp_c:

Builder(menu='zoom', metabolite_data={'aspsa_c': 1.0503965108821234e-07, 'nadp_c': 6.800000012601299, 'pi_c': …

# Mixed Objective

In [6]:
# Constrains various branches
kinetic_targets = {"ASPTA": -4.0, "ASAD": -3.5, "DAPE": 1.7}
# Constrains one remaining branch
production_targets = {"met__L_c": 0.2}
# Add non-objective endpoints as exchanges
exchanges=['5mthf_c', 'adp_c', 'akg_c', 'atp_c', 'co2_c', 'coa_c',
           'cys__L_c', 'glu__L_c', 'h2o_c', 'h_c', 'nadp_c', 'nadph_c',
           'nh4_c', 'oaa_c', 'pi_c', 'pyr_c', 'succ_c', 'succoa_c', 'thf_c',
           'asp__L_c', 'lys__L_c', 'thr__L_c'
          ]

# Production targets and kinetic objectives are added as named objectives.
fba2 = fba_gd.GradientDescentFba(
    reactions=reactions,
    exchanges=exchanges,
    target_metabolites=production_targets.keys(),
)
fba2.add_objective("homeostatic",
                   fba_gd.TargetDmdtObjective(fba2.network, production_targets.keys()))
fba2.add_objective("kinetic",
                   fba_gd.TargetVelocityObjective(fba2.network, kinetic_targets.keys()))


In [7]:
# At solve time we pass the current targets for production and kinetic objectives. Again
# the details for how these are passed can be reworked.

# We also pass in the previous solution as the initial flux values, to show how it's done.
previous = soln1.velocities

soln2 = fba2.solve({"homeostatic": production_targets, "kinetic": kinetic_targets},
                   initial_velocities=previous)
print(f"RMSE = {np.sqrt(np.sum(np.square(soln2.ss_residual)))}")


RMSE = 1.8776943518128583e-08


In [8]:
print("Velocities")
for reaction_id in sorted(reaction["reaction id"] for reaction in reactions):
    if reaction_id in kinetic_targets:
        print(f"  {reaction_id:>10}:   {soln2.velocities[reaction_id]:+.2f} / {kinetic_targets[reaction_id]:+.2f}")
    else:
        print(f"  {reaction_id:>10}:   {soln2.velocities[reaction_id]:+.2f}")

print()
print("Production Targets")
for molecule in sorted(production_targets):
    print(f"  {molecule:>10}:   {soln2.dm_dt[molecule]:.2f} / {production_targets[molecule]:.2f}")

print()
print("Exchanges")
for molecule in sorted(exchanges):
    print(f"  {molecule:>10}:   {soln2.dm_dt[molecule]:+.2f}")

print()
print("Intermediates")
for molecule in fba2.network.molecule_ids():
    if molecule not in production_targets and molecule not in exchanges:
        print(f"  {molecule:>10}:   {soln2.dm_dt[molecule]: .2e}")

escher.Builder(map_json='FBAgd-demo-eschermap.json',
               menu='zoom',
               never_ask_before_quit=True,
               reaction_data=soln2.velocities,
               reaction_styles=['color', 'size', 'text', 'abs'],
               metabolite_data=soln2.dm_dt,
               metabolite_styles=['color', 'text'],
               metabolite_scale_preset='GeGaRd',
              )


Velocities
        ASAD:   -3.50 / -3.50
        ASPK:   +3.50
       ASPTA:   -4.00 / -4.00
       CYSTL:   +0.20
       DAPDC:   +1.70
        DAPE:   +1.70 / +1.70
      DHDPRy:   +1.70
       DHDPS:   +1.70
        HSDy:   -1.80
         HSK:   +1.60
        HSST:   +0.20
        METS:   +0.20
       SDPDS:   +1.70
       SDPTA:   -1.70
       SHSL1:   +0.20
       THDPS:   +1.70
        THRS:   +1.60

Production Targets
    met__L_c:   0.20 / 0.20

Exchanges
     5mthf_c:   -0.20
       adp_c:   +5.10
       akg_c:   +5.70
    asp__L_c:   +0.50
       atp_c:   -5.10
       co2_c:   +1.70
       coa_c:   +1.90
    cys__L_c:   -0.20
    glu__L_c:   -5.70
       h2o_c:   -1.80
         h_c:   -5.00
    lys__L_c:   +1.70
      nadp_c:   +7.00
     nadph_c:   -7.00
       nh4_c:   +0.20
       oaa_c:   -4.00
        pi_c:   +5.10
       pyr_c:   -1.50
      succ_c:   +1.90
    succoa_c:   -1.90
       thf_c:   +0.20
    thr__L_c:   +1.60

Intermediates
     aspsa_c:    7.51e-10
     4p

Builder(menu='zoom', metabolite_data={'aspsa_c': 7.505347454639377e-10, 'nadp_c': 6.99999999699786, 'pi_c': 5.…